In [ ]:
Effect of water on the dry reforming
#甲烷和二氧化碳的量都保持不变
#通过调整不同的二氧化碳的初始加入量来看二氧化碳对干重整的影响

Effect of water on dry reforming of methane
# The amount of methane and carbon dioxide remains the same
# The effect of carbon dioxide on dry reforming is seen by adjusting the initial amount of different carbon dioxide

In [ ]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd

def gibbs_equation(T):
    delta_G1 = 86250 - 107.64 * T
    K1 = np.exp(-delta_G1 / (8.314 * T))
    
    delta_G2 = 170700 - 174.46 * T
    K2 = np.exp(-delta_G2 / (8.314 * T))
    
    delta_G3 = 137100 - 145.06 * T
    K3 = np.exp(-delta_G3 / (8.314 * T))
       
    return K1, K2, K3

def calculate_equilibrium_constants(variables, T, P, N_ch4, N_h2o, N_co2, K1, K2, K3):
    x, y, z = variables
    
    p1 = P / 101300
    
    E_ch4 = N_ch4 - x
    E_h2o = N_h2o - z
    E_h2 = 2*x+z
    E_co2 = N_co2 - y
    E_co = 2 * y +z
    E_c = x-y-z
    
    E_total = E_ch4 + E_h2o + E_co2 + E_h2 + E_co 
    
    x_ch4 = E_ch4 / E_total
    x_h2o = E_h2o / E_total
    x_h2 = E_h2 / E_total
    x_co2 = E_co2 / E_total
    x_co = E_co / E_total
    x_c = E_c
    
    eq1 = (x_h2*x_h2 * p1)  / (x_ch4 )  - K1
    eq2 = (x_co *x_co* p1)  / (x_co2 )  - K2
    eq3 = (x_co*x_h2 * p1) /  (x_h2o )  - K3

    return np.array([eq1, eq2, eq3])

# 检查约束条件的函数
# Function to check constraints
def check_constraints(E_co, E_c, E_h2, E_h2o, E_ch4, E_co2, x, y, z, N_co2, N_h2o, N_ch4, T, K11, K22, K33):
    return (E_co > 0 
            and E_c >= 0 
            and E_h2 > 0 
            #and E_h2o >=0 
            #and E_ch4 >= 0 
            #and E_co2 >= 0 
            and x > 0 
            #and y > 0
            #and z > 0 
            and ((T == 800 and 0.9 <= K11 <= 1.1 and 0.008 <= K22 <= 0.012 and 0.03 <= K33 <= 0.06) or
                 (T == 850 and 2 <= K11 <= 2.3 and 0.03 <= K22 <= 0.05 and 0.12 <= K33 <= 0.16) or
                 (T == 900 and 4 <= K11 <= 4.3 and 0.15 <= K22 <= 0.18 and 0.3 <= K33 <= 0.5) or
                 (T == 950 and 7.3 <= K11 <= 7.8 and 0.5 <= K22 <= 0.6 and 0.9 <= K33 <= 1.2) or
                 (T == 1000 and 12 <= K11 <= 14 and 1.4<= K22 <= 1.8 and 2.4 <= K33 <= 3) or
                 (T == 1050 and 20 <= K11 <= 23 and 4<= K22 <= 4.6 and 5.3 <= K33 <= 6) or
                 (T == 1100 and 31 <= K11 <= 35 and 9 <= K22 <= 11 and 10 <= K33 <= 13) or
                 (T == 1150 and 50 <= K11 <= 52 and 22 <= K22 <= 24 and 22 <= K33 <= 24) or
                 (T == 1200 and 70 <= K11 <= 77 and 45 <= K22 <= 50 and 39 <= K33 <= 43) or
                 (T == 1250 and 100 <= K11 <= 110 and 93 <= K22 <= 97 and 68 <= K33 <= 72) or
                 (T == 1300 and 135 <= K11 <= 149 and 165 <= K22 <= 195 and 105 <= K33 <= 129) or
                 (T == 1350 and 185 <= K11 <= 199 and 315 <= K22 <= 330 and 175 <= K33 <= 199) or
                 (T == 1400 and 240 <= K11 <= 269 and 540 <= K22 <= 569 and 269 <= K33 <= 309) or
                 (T == 1450 and 317 <= K11 <= 337 and 909 <= K22 <= 929 and 420 <= K33 <= 454) or
                 (T == 1500 and 396 <= K11 <= 437 and 1374 <= K22 <= 1575 and 615 <= K33 <= 656) ))

# 创建一个空的 DataFrame 来存储结果
# Create an empty DataFrame to store the result
columns = ['T', 'P', 'N_ch4', 'N_h2o', 'N_co2', 'x', 'y', 'z', 'E_ch4', 'E_h2o', 'E_co2', 'E_h2', 'E_co', 'E_c', 'E_total']
results_df = pd.DataFrame(columns=columns)


#######通过调整不同的二氧化碳的初始加入量来看二氧化碳对干重整的影响
####### The effect of carbon dioxide on dry reforming is seen by adjusting the initial addition of different carbon dioxide
# 生成 N_h2o_range 和 N_co2_range
# Generate N_h2o_range and N_co2_range
N_h2o_range = [i * 0.1 for i in range(1, 2)]
N_co2_range = [i * 0.1 for i in range(10, 11)]

T_range = range(800, 1301, 100)
P_range = range(100000, 1000001, 400000)
N_ch4_range = range(1, 2, 1)

# 对所有的量进行 for 循环求解
# for loop solution for all quantities
for T in T_range:
    for P in P_range:
        for N_ch4 in N_ch4_range:
             for N_co2 in N_co2_range:
                 for N_h2o in N_h2o_range:
                
                    # 标志，表示是否已经找到结果
                    # flag indicates whether a result has been found
                    found_result = False
                    
                    # 对于每组 T，P，N_ch4，N_h2o，N_co2 的值，计算所有可能的初始猜测值的结果
                    # For each set of values T, P, N_ch4, N_h2o, N_co2, calculate the result of all possible initial guesses
                    for x_guess in np.arange(-0.05, 1.01, 0.01):
                        for y_guess in np.arange(-0.05, 1.01, 0.01):
                            for z_guess in np.arange(-2, 0.1, 0.01):
                                initial_guess = [x_guess, y_guess, z_guess]
                                K1, K2, K3 = gibbs_equation(T)
                                result = fsolve(calculate_equilibrium_constants, initial_guess, args=(T, P, N_ch4, N_h2o, N_co2, K1, K2, K3), xtol=1e-7, maxfev=100000)
                                
                                # 处理结果并添加到 DataFrame
                                # Process the result and add it to the DataFrame
                                #x, y, z = np.around(result, 4)  # 保留六位小数，这是考虑k值大小的思考，也是数据太小了，所以取有效位数
                                x = np.around(result[0], 9)  
                                y = np.around(result[1], 9)  
                                z = np.around(result[2], 9)  
                                p1 = P / 101300
              
                                E_ch4 = N_ch4 - x
                                E_h2o = N_h2o - z
                                E_h2 = 2*x+z
                                E_co2 = N_co2 - y
                                E_co = 2 * y +z
                                E_c = x-y-z
                                E_total = E_ch4 + E_h2o + E_co2 + E_h2 + E_co 
    
                                x_ch4 = E_ch4 / E_total
                                x_h2o = E_h2o / E_total
                                x_h2 = E_h2 / E_total
                                x_co2 = E_co2 / E_total
                                x_co = E_co / E_total
                                x_c = E_c
    
                                K11 = (x_h2*x_h2 * p1)  / (x_ch4 )  
                                K22 = (x_co *x_co* p1)  / (x_co2 ) 
                                K33 = (x_co*x_h2 * p1) /  (x_h2o )  
                                
                                # 添加限制条件
                                # Add restrictions
                                if check_constraints(E_co, E_c, E_h2, E_h2o, E_ch4, E_co2, x, y, z, N_co2, N_h2o, N_ch4, T, K11, K22, K33):
                                   
                                    print(f'T: {T}, P: {P}, N_ch4: {N_ch4}, N_h2o: {N_h2o}, N_co2: {N_co2}, x: {x}, y: {y}, z: {z}, E_total: {E_total}')
                                    print(f'E_ch4: {E_ch4}, E_h2o: {E_h2o}, E_co2: {E_co2}, E_h2: {E_h2}, E_co: {E_co}, E_c: {E_c}')

                                    # 将结果添加到 DataFrame
                                    new_row = pd.DataFrame([{
                                               'T': T, 'P': P, 'N_ch4': N_ch4, 'N_h2o': N_h2o, 'N_co2': N_co2,
                                               'x': x, 'y': y, 'z': z,
                                               'E_ch4': E_ch4, 'E_h2o': E_h2o, 'E_co2': E_co2, 'E_h2': E_h2, 'E_co': E_co, 'E_c': E_c, 'E_total': E_total
                                                 }])
                                    results_df = pd.concat([results_df, new_row], ignore_index=True)
                                    
                                    
                                    found_result = True
                                    
                                    # 找到结果后停止计算其他初始猜测值，进行下一个 T，P，N_ch4，N_h2o，N_co2 的循环
                                    # After finding the result, stop calculating other initial guesses 
                                    #and proceed to the next cycle of T, P, N_ch4, N_h2o, N_co2
                                    break
                                    
                                # 如果已经找到结果，则直接跳出内部循环
                                # If the result is found, then jump out of the inner loop
                                
                                if found_result:
                                    break
                            if found_result:
                                break
                        if found_result:
                            break
                    if found_result:
                        break

# 将 DataFrame 保存为 Excel 文件
# Save the DataFrame as an Excel file
results_df.to_excel('CO2-CH4-H2O-1-1-0.1-output.xlsx', index=False)


In [ ]:
二氧化碳对甲烷蒸汽的影响
#甲烷和水的量都保持不变
#通过调整不同的二氧化碳的初始加入量来看二氧化碳对甲烷蒸汽重整的影响

Effect of carbon dioxide on methane vapor
# The amount of methane and water remains the same
# The effect of carbon dioxide on methane steam reforming is seen by adjusting the initial addition of different carbon dioxide

In [ ]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd

def gibbs_equation(T):
    delta_G1 = 86250 - 107.64 * T
    K1 = np.exp(-delta_G1 / (8.314 * T))
    
    delta_G2 = 170700 - 174.46 * T
    K2 = np.exp(-delta_G2 / (8.314 * T))
    
    delta_G3 = 137100 - 145.06 * T
    K3 = np.exp(-delta_G3 / (8.314 * T))
       
    return K1, K2, K3

def calculate_equilibrium_constants(variables, T, P, N_ch4, N_h2o, N_co2, K1, K2, K3):
    x, y, z = variables
    
    p1 = P / 101300
    
    E_ch4 = N_ch4 - x
    E_h2o = N_h2o - z
    E_h2 = 2*x+z
    E_co2 = N_co2 - y
    E_co = 2 * y +z
    E_c = x-y-z
    
    E_total = E_ch4 + E_h2o + E_co2 + E_h2 + E_co 
    
    x_ch4 = E_ch4 / E_total
    x_h2o = E_h2o / E_total
    x_h2 = E_h2 / E_total
    x_co2 = E_co2 / E_total
    x_co = E_co / E_total
    x_c = E_c
    
    eq1 = (x_h2*x_h2 * p1)  / (x_ch4 )  - K1
    eq2 = (x_co *x_co* p1)  / (x_co2 )  - K2
    eq3 = (x_co*x_h2 * p1) /  (x_h2o )  - K3

    return np.array([eq1, eq2, eq3])


def check_constraints(E_co, E_c, E_h2, E_h2o, E_ch4, E_co2, x, y, z, N_co2, N_h2o, N_ch4, T, K11, K22, K33):
    return (E_co > 0 
            and E_c >= 0 
            and E_h2 > 0 
            #and E_h2o >=0 
            #and E_ch4 >= 0 
            #and E_co2 >= 0 
            and x > 0 
            #and y > 0
            #and z > 0 
            and ((T == 800 and 0.9 <= K11 <= 1.1 and 0.008 <= K22 <= 0.012 and 0.03 <= K33 <= 0.06) or
                 (T == 850 and 2 <= K11 <= 2.3 and 0.03 <= K22 <= 0.05 and 0.12 <= K33 <= 0.16) or
                 (T == 900 and 4 <= K11 <= 4.3 and 0.15 <= K22 <= 0.18 and 0.3 <= K33 <= 0.5) or
                 (T == 950 and 7.3 <= K11 <= 7.8 and 0.5 <= K22 <= 0.6 and 0.9 <= K33 <= 1.2) or
                 (T == 1000 and 12 <= K11 <= 14 and 1.4<= K22 <= 1.8 and 2.4 <= K33 <= 3) or
                 (T == 1050 and 20 <= K11 <= 23 and 4<= K22 <= 4.6 and 5.3 <= K33 <= 6) or
                 (T == 1100 and 31 <= K11 <= 35 and 9 <= K22 <= 11 and 10 <= K33 <= 13) or
                 (T == 1150 and 50 <= K11 <= 52 and 22 <= K22 <= 24 and 22 <= K33 <= 24) or
                 (T == 1200 and 70 <= K11 <= 77 and 45 <= K22 <= 50 and 39 <= K33 <= 43) or
                 (T == 1250 and 100 <= K11 <= 110 and 93 <= K22 <= 97 and 68 <= K33 <= 72) or
                 (T == 1300 and 135 <= K11 <= 149 and 165 <= K22 <= 195 and 105 <= K33 <= 129) or
                 (T == 1350 and 185 <= K11 <= 199 and 315 <= K22 <= 330 and 175 <= K33 <= 199) or
                 (T == 1400 and 240 <= K11 <= 269 and 540 <= K22 <= 569 and 269 <= K33 <= 309) or
                 (T == 1450 and 317 <= K11 <= 337 and 909 <= K22 <= 929 and 420 <= K33 <= 454) or
                 (T == 1500 and 396 <= K11 <= 437 and 1374 <= K22 <= 1575 and 615 <= K33 <= 656) ))


columns = ['T', 'P', 'N_ch4', 'N_h2o', 'N_co2', 'x', 'y', 'z', 'E_ch4', 'E_h2o', 'E_co2', 'E_h2', 'E_co', 'E_c', 'E_total']
results_df = pd.DataFrame(columns=columns)


N_h2o_range = [i * 0.1 for i in range(10, 11)]
N_co2_range = [i * 0.1 for i in range(1, 2)]

T_range = range(800, 1301, 100)
P_range = range(100000, 1000001, 400000)
N_ch4_range = range(1, 2, 1)


for T in T_range:
    for P in P_range:
        for N_ch4 in N_ch4_range:
             for N_co2 in N_co2_range:
                 for N_h2o in N_h2o_range:
                
                    
                    found_result = False
                    
                    
                    for x_guess in np.arange(-0.5, 1.01, 0.01):
                        for y_guess in np.arange(-0.51, 0.1, 0.01):
                            for z_guess in np.arange(-0.2, 1.01, 0.01):
                                initial_guess = [x_guess, y_guess, z_guess]
                                K1, K2, K3 = gibbs_equation(T)
                                result = fsolve(calculate_equilibrium_constants, initial_guess, args=(T, P, N_ch4, N_h2o, N_co2, K1, K2, K3), xtol=1e-7, maxfev=100000)
                                
                                
                                #x, y, z = np.around(result, 4)  
                                x = np.around(result[0], 9)  
                                y = np.around(result[1], 9)  
                                z = np.around(result[2], 9)  
                                p1 = P / 101300
              
                                E_ch4 = N_ch4 - x
                                E_h2o = N_h2o - z
                                E_h2 = 2*x+z
                                E_co2 = N_co2 - y
                                E_co = 2 * y +z
                                E_c = x-y-z
                                E_total = E_ch4 + E_h2o + E_co2 + E_h2 + E_co 
    
                                x_ch4 = E_ch4 / E_total
                                x_h2o = E_h2o / E_total
                                x_h2 = E_h2 / E_total
                                x_co2 = E_co2 / E_total
                                x_co = E_co / E_total
                                x_c = E_c
    
                                K11 = (x_h2*x_h2 * p1)  / (x_ch4 )  
                                K22 = (x_co *x_co* p1)  / (x_co2 ) 
                                K33 = (x_co*x_h2 * p1) /  (x_h2o )  
                                
                               
                                if check_constraints(E_co, E_c, E_h2, E_h2o, E_ch4, E_co2, x, y, z, N_co2, N_h2o, N_ch4, T, K11, K22, K33):
                                    
                                    print(f'T: {T}, P: {P}, N_ch4: {N_ch4}, N_h2o: {N_h2o}, N_co2: {N_co2}, x: {x}, y: {y}, z: {z}, E_total: {E_total}')
                                    print(f'E_ch4: {E_ch4}, E_h2o: {E_h2o}, E_co2: {E_co2}, E_h2: {E_h2}, E_co: {E_co}, E_c: {E_c}')

                                    
                                    new_row = pd.DataFrame([{
                                               'T': T, 'P': P, 'N_ch4': N_ch4, 'N_h2o': N_h2o, 'N_co2': N_co2,
                                               'x': x, 'y': y, 'z': z,
                                               'E_ch4': E_ch4, 'E_h2o': E_h2o, 'E_co2': E_co2, 'E_h2': E_h2, 'E_co': E_co, 'E_c': E_c, 'E_total': E_total
                                                 }])
                                    results_df = pd.concat([results_df, new_row], ignore_index=True)
                                    
                                    
                                    found_result = True
                                    
                                    
                                    break
                                    
                               
                                if found_result:
                                    break
                            if found_result:
                                break
                        if found_result:
                            break
                    if found_result:
                        break


results_df.to_excel('CO2-CH4-H2O-0.1-1-1-output.xlsx', index=False)
